In [ ]:
# ベースとなる最初のデータを取得
import requests
from datetime import datetime
import time
import json

chart_sec = 60         # 保存したいローソク足の時間軸
file = "./test.json"   # 保存するファイル名

# Cryptowatchのデータを単に保存するだけの関数
def get_data(min, path, before=0, after=0):
	
	# APIで価格データを取得
	params = {"periods" : min }
	if before != 0:
		params["before"] = before
	if after != 0:
		params["after"] = after
	response = requests.get("https://api.cryptowat.ch/markets/bitflyer/btcfxjpy/ohlc",params)
	data = response.json()
	
	#ファイルに書き込む
	file = open( path,"w",encoding="utf-8")
	json.dump(data,file)
	
	return data

# メイン処理
get_data(chart_sec, file ,after=1483228800)

In [91]:
import requests
from datetime import datetime
from pprint import pprint
import time
import json

# データ取得のエラー送信ためのLineアクセストークン
line_token = 'Fk857wFigGPmcnSNTyjR2Ed0Olz2pxfIDGEUxFPMVRU'

def line_notify(text):
    url = 'https://notify-api.line.me/api/notify'
    data = {'message': text}
    headers = {'Authorization': 'Bearer ' + line_token}
    requests.post(url, data=data, headers=headers)

def accumulate_diff_data(min, read_path, save_path, before = 0, after = 0):
    
    params = {"periods": min }
    if before:
	    params["before"] = before
    if after:
	    params["after"] = after
    cnt = 0
    while True:
        try: 
            response = requests.get("https://api.cryptowat.ch/markets/bitflyer/btcfxjpy/ohlc",params)
            web_data = response.json()
            web_data_set = set(map(tuple, web_data["result"][str(min)]))
            break
        except requests.exceptions.RequestException as e:
            print('通信でエラーがあったため、３分待ってやり直します :', e)
            if cnt <= 20:
                time.sleep(180)
                cnt += 1
            else:
                print('通信エラーが解消しないため、データ取得の実行を終了します :', e)
                line_notify('通信エラーが解消しないため、データ取得の実行を終了します :')
                return

    try:
        f = open(read_path ,"r", encoding="utf-8")
    # 累積ファイルがないときの例外処理
    except FileNotFoundError:
        print('既存ファイルがないので、新しいファイルを作成します')
        f = open(read_path ,"w", encoding="utf-8")
        json.dump(web_data,f)
        if len(list(web_data_set)):
            data_len = len(list(web_data_set))
            print(f"{data_len}件のデータがありました")
            pprint(list(web_data_set))
        return web_data # 戻り値なしでもOK

    f_data = json.load(f)
    del f_data["result"][str(min)][-1] # 末尾は被るので削除(リアルタイム形成中のデータが含まれる)
    f_data_set = set(map(tuple, f_data["result"][str(min)]))
    # 差分をとる
    diff_data_set = web_data_set - f_data_set
    diff_data = list(diff_data_set)

    if len(diff_data):
        data_len = len(diff_data)
        print(f"{data_len}件の追加データがありました")
        diff_data.sort(key = lambda x:x[0])
        f_data["result"][str(min)].extend(diff_data)
        pprint(diff_data)

    f = open(save_path, "w", encoding="UTF-8")
    json.dump(f_data, f)

    return f_data # 戻り値なしでもOK



In [ ]:
# データ取得＆累積のメインコード

# 第２、３引数のファイル名を同じにすることで、上書き
chart_sec_list = ['60', '180', '300', '900', '1800', '3600', '7200', '14400', '21600', '43200', '86400', '259200', '604800', '604800_Monday']

chart_sec = 60
file_1 = "./test.json"
file_2 = "./test_2.json"
file_n = './604800.json'

# for chart_sec in chart_sec_list:
#     file_name = f'data_{chart_sec}.json'

accumulate_diff_data(chart_sec, file_2, file_2, after=1483228800)


In [92]:
chart_sec = 60
file_2 = "./test_3.json"
accumulate_diff_data(chart_sec, file_2, file_2, after=1483228800)

既存ファイルがないので、新しいファイルを作成します
6000件のデータがありました
[(1643938980, 4302136, 4302984, 4301253, 4301627, 0.8925, 3839426.05),
 (1644043380,
  4821500,
  4822213,
  4816328,
  4820415,
  10.51092361,
  50652764.27335591),
 (1644075180, 4848324, 4852000, 4845383, 4852000, 4.9781733, 24136990.38227434),
 (1643879400, 4283678, 4285047, 4283662, 4284224, 1.059, 4536955.26434028),
 (1643737320, 4489271, 4495746, 4487639, 4487639, 3.95249871, 17754072.38219),
 (1643938860, 4302269, 4302269, 4301271, 4301620, 0.81, 3484570.69),
 (1643773800, 4468027, 4468460, 4466212, 4468460, 0.98000434, 4377938.58724459),
 (1643804220, 4462998, 4464414, 4460570, 4461753, 2.06416282, 9212279.98067078),
 (1644077880,
  4830000,
  4839545,
  4827662,
  4834875,
  6.65893036,
  32197806.30264461),
 (1643937960, 4315337, 4315651, 4314111, 4314978, 0.77676158, 3351865.65924328),
 (1643823480,
  4307277,
  4309377,
  4306640,
  4309377,
  2.57183211,
  11080116.19684301),
 (1643938140, 4315706, 4317120, 4315148, 4315306, 0.27, 

{'result': {'60': [[1643731080,
    4465387,
    4471231,
    4463140,
    4471231,
    8.29926238,
    37077052.96870288],
   [1643731140,
    4470900,
    4491418,
    4469412,
    4475445,
    17.96572144,
    80425794.35781664],
   [1643731200,
    4476201,
    4493082,
    4473136,
    4479400,
    21.17135089,
    94867097.22518279],
   [1643731260,
    4479400,
    4484876,
    4478310,
    4483282,
    10.64658701,
    47719593.89991305],
   [1643731320,
    4483182,
    4485990,
    4480683,
    4483601,
    7.45005,
    33399615.35009684],
   [1643731380,
    4483509,
    4485385,
    4480180,
    4482282,
    7.97199638,
    35730629.02994501],
   [1643731440,
    4481869,
    4485646,
    4479095,
    4482761,
    6.58378888,
    29509693.04101801],
   [1643731500,
    4482829,
    4485896,
    4478416,
    4480796,
    13.76847804,
    61705805.51881147],
   [1643731560,
    4481089,
    4484172,
    4475489,
    4475925,
    9.17483911,
    41092492.21334506],
   [1643731

In [55]:
# 日付チェック
import requests
from datetime import datetime
from pprint import pprint
import time
import json

def op_file(path):
    with open(path, encoding='utf-8') as f:
        data = json.load(f)
        print('開始日時: ', datetime.fromtimestamp(data['result']['60'][0][0]).strftime('%Y/%m/%d %H:%M'))
        print('終了日時: ', datetime.fromtimestamp(data['result']['60'][-1][0]).strftime('%Y/%m/%d %H:%M'))
    return

op_file('test_2.json')

開始日時:  2022/01/30 10:22
終了日時:  2022/02/05 05:39


In [58]:
# 累積ファイルからデータを取り出して成形
import requests
from datetime import datetime
import time
import json
from pprint import pprint

chart_sec = 60 
file = "./test2.json" 
start_period = "2018/05/07 00:00"
end_period = "2018/05/08 00:00"

def get_price_from_file( min, path, start_period = None, end_period = None):
	f = open(path,"r", encoding="UTF-8")
	data = json.load(f)
	
	start_unix = 0
	end_unix = 9999999999
	
	if start_period:
		start_period = datetime.strptime(start_period,"%Y/%m/%d %H:%M")
		start_unix = int(start_period.timestamp())
	if end_period:
		end_period = datetime.strptime(end_period,"%Y/%m/%d %H:%M")
		end_unix = int(end_period.timestamp())
	
	price = []
	for i in data["result"][str(min)]:
		if i[0] >= start_unix and i[0] <= end_unix:
			if i[1] and i[2] and i[3] and i[4]:
				price.append({
					"close_time": i[0],
					"close_time_dt": datetime.fromtimestamp(i[0]).strftime("%Y/%m/%d %H:%M"),
					"open_price": i[1],
					"high_price": i[2],
					"low_price": i[3],
					"close_price": i[4]
					})
	
	return price

In [ ]:
data = accumulate_diff_data(chart_sec, file_1, file_2)
print(data)